<a href="https://colab.research.google.com/github/arumajirou/-daily-test/blob/main/Meta's_OPT_IML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 68.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 101.4 MB/s eta 0:00:00


Using Hugging Face `transformers` with the usual code

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

- PyTorchの自然言語処理(NLP)タスクのライブラリであるtransformersライブラリのpythonインポートです。インポートは以下の通りです。

- AutoModelForCausalLM: テキストのシーケンスを生成するために事前にトレーニングされた言語モデルをロードするためのクラスです。
- AutoTokenizer。言語モデルで使用されるトークナイザーを読み込むためのクラスです．
- torch PyTorchは、機械学習モデルの構築と学習に使用される深層学習フレームワークです。

In [3]:
model = AutoModelForCausalLM.from_pretrained("facebook/opt-iml-max-1.3b", torch_dtype=torch.float16).cuda()

# the fast tokenizer currently does not work correctly
tokenizer = AutoTokenizer.from_pretrained("facebook/opt-iml-max-1.3b", use_fast=False)

- 上記のコードはHugging FaceのTransformersライブラリを使用して、「facebook/opt-iml-max-1.3b」という事前に学習された言語モデルをロードしています。
- 言語モデルはtorch_dtypeがfloat16でロードされ、学習時に16ビット浮動小数点数を使用することを意味し、".cuda() "を使用してGPUにロードされます。
- 対応するトークナイザーもロードされ、「use_fast」パラメータはFalseに設定されています。
- これは、トークナイザーが高速トークナイゼーション手法を使用しないことを意味します。

In [4]:
prompt = """Write a Tweet with Hashtags about Elon Musk:"""

In [5]:
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.cuda()

generated_ids = model.generate(input_ids,)

output = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)


/usr/local/lib/python3.8/dist-packages/transformers/generation/utils.py:1273: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 20 (`generation_config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


- このコードは Python で書かれており、transformersライブラリを使用しています。
- Facebook のopt-iml-max-1.3bという変換言語モデルと、それに関連するトークナイザを初期化します。
- トークナイザーは、高速トークナイザーに関する既知の問題のため、use_fast=Falseで初期化されます。
- このコードは、プロンプトをトークナイザーで入力IDにエンコードし、言語モデルを用いて入力IDから新しいテキストを生成し、生成されたIDをトークナイザーでテキストにデコードして、プロンプトが与えられたときのテキストを生成するものである。
- 生成されたテキストは出力変数に格納される．

In [6]:
print(output[0])

Write a Tweet with Hashtags about Elon Musk:

Tweet #ElonMusk #


Using Hugging Face Pipeline

In [7]:
from transformers import pipeline

generator = pipeline('text-generation', model="facebook/opt-iml-max-1.3b", device = 0)


In [8]:

generator("""Given a text, you need to extract Named Entities like Place. 
Input: America and India are really expensive places.
Output:""", max_length = 100)[0]['generated_text']


'Given a text, you need to extract Named Entities like Place. \nInput: America and India are really expensive places.\nOutput: America'

- 上記のコードでは、transformers ライブラリから「テキスト生成」パイプラインをインポートし、「facebook/opt-iml-max-1.3b」モデルと使用するデバイス（デバイス 0）を指定してパイプラインをインスタンス化する。
- そして、生成されたパイプラインを使って、与えられたプロンプトに基づいたテキストを生成する。プロンプトは「generator」関数の引数として渡され、生成されるテキストの最大長は100に設定される。
- 生成されたテキストは辞書として返され、キー "generated_text "に生成されたテキストが格納される。

- テキストベースの質問に応答し、情報を提供する
- 与えられたプロンプトに対応するテキストを生成する
- テキストをトークン化し、テンソルにエンコードする
- トークン化されたテンソルをテキストにデコードして戻す
具体的な例では、どのようにすればよいかを示しました。

- 変換ライブラリから「facebook/opt-iml-max-1.3b」言語モデルとそれに対応するトークナイザーを読み込み、使用する。
- イーロン・マスクに関するツイートのプロンプトに応じたテキストを生成する
- 場所のようなNamed Entityを抽出するよう求めるプロンプトに応答して、テキストを生成する。
他に何かありましたら教えてください。

- FacebookのOML-MAX 1.3Bのような事前に学習させた言語モデルを用いたテキスト生成は、入力プロンプトに基づくテキスト生成が必要な様々な場面で有用です。
- 具体的なユースケースをいくつか紹介する。

1. コンテンツ作成。与えられたトピックやタイトルをもとに、記事、ブログ記事、商品説明などを自動生成すること。

2. チャットボット。ユーザーの問い合わせに自然言語で応答できる会話型エージェントを構築すること。

3. 文章補完。部分的に書かれた文章を自動的に完成させること。

4. テキストの要約。長い論文やレポートを短い文章に要約すること。

5. マーケティングと広告。商品説明や広告を自動生成。

6. ソーシャルメディアソーシャルメディア用のキャプション、ハッシュタグ、投稿を自動生成します。

- これらは、テキスト生成が利用できる幅広いアプリケーションのほんの一例に過ぎない。
- 大規模な事前学習済み言語モデルの登場により、生成されるテキストの品質は大幅に向上し、これらのアプリケーションの実現可能性はさらに高まっています。